In [3]:
import urllib.request
import os

# 1. Prototxt (network definition)
pt_url    = "https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/deploy.prototxt"
pt_path   = "MobileNetSSD_deploy.prototxt"
if not os.path.isfile(pt_path):
    urllib.request.urlretrieve(pt_url, pt_path)

# 2. Weights (the pretrained Caffe model)
model_url  = "https://github.com/chuanqi305/MobileNet-SSD/raw/master/mobilenet_iter_73000.caffemodel"
model_path = "MobileNetSSD_deploy.caffemodel"
if not os.path.isfile(model_path):
    urllib.request.urlretrieve(model_url, model_path)

print("Files ready:", pt_path, model_path)

Files ready: MobileNetSSD_deploy.prototxt MobileNetSSD_deploy.caffemodel


In [11]:
import cv2
import numpy as np
import pandas as pd

# ─── Settings ────────────────────────────────────────────────────────────────
INPUT_VIDEO      = "man-sleeping.mp4"
OUTPUT_VIDEO     = "man-sleeping_motion_bbox.mp4"
OUTPUT_CSV       = "man-sleeping_motion_segments.csv"
MIN_CONTOUR_AREA = 800    # smallest ripple to count as “movement”
FPS_SMOOTH       = 3      # median‐filter window for state smoothing

# ─── Video I/O ───────────────────────────────────────────────────────────────
cap   = cv2.VideoCapture(INPUT_VIDEO)
fps   = cap.get(cv2.CAP_PROP_FPS)
w     = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h     = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out    = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (w, h))

# ─── Background model ────────────────────────────────────────────────────────
backSub = cv2.createBackgroundSubtractorMOG2(
    history=500, varThreshold=16, detectShadows=False)

# ─── Helpers for smoothing ────────────────────────────────────────────────────
state_history = []
def smooth_state(new_state):
    state_history.append(new_state)
    if len(state_history) > FPS_SMOOTH:
        state_history.pop(0)
    # return majority vote
    return max(set(state_history), key=state_history.count)

# ─── Segments bookkeeping ─────────────────────────────────────────────────────
segments      = []
current_state = None
current_start = None
frame_idx     = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    t = frame_idx / fps  # time in seconds

    # 1) Compute FG mask and clean it up
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fg   = backSub.apply(gray)
    fg   = cv2.morphologyEx(
        fg,
        cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    )

    # 2) Find contours & pick the largest
    cnts, _ = cv2.findContours(
        fg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    box = None
    if cnts:
        # area of each contour
        areas = [cv2.contourArea(c) for c in cnts]
        max_i = np.argmax(areas)
        if areas[max_i] > MIN_CONTOUR_AREA:
            box = cv2.boundingRect(cnts[max_i])  # x,y,w,h

    # 3) Determine state & annotate
    raw_state = "Moving" if box else "Stationary"
    state     = smooth_state(raw_state)

    if box:
        x,y,wb,hb = box
        color = (0,0,255) if state=="Moving" else (0,255,0)
        cv2.rectangle(frame, (x,y), (x+wb,y+hb), color, 2)
        cv2.putText(frame, state, (x, max(y-10,0)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    else:
        # Optionally draw “no motion” text
        cv2.putText(frame, state, (20,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    # 4) Record transitions
    if current_state is None:
        current_state = state
        current_start = t
    elif state != current_state:
        segments.append({
            "state":    current_state,
            "start_s":  round(current_start, 2),
            "end_s":    round(t, 2),
            "duration": round(t-current_start, 2)
        })
        current_state = state
        current_start = t

    # 5) Write frame & advance
    out.write(frame)
    frame_idx += 1

# final segment
if current_state is not None:
    end_t = frame_idx / fps
    segments.append({
        "state":    current_state,
        "start_s":  round(current_start, 2),
        "end_s":    round(end_t, 2),
        "duration": round(end_t-current_start, 2)
    })

# ─── Save CSV ────────────────────────────────────────────────────────────────
df = pd.DataFrame(segments)
df.to_csv(OUTPUT_CSV, index=False)
print("Wrote:", OUTPUT_CSV, OUTPUT_VIDEO)

cap.release()
out.release()
cv2.destroyAllWindows()


Wrote: man-sleeping_motion_segments.csv man-sleeping_motion_bbox.mp4
